# I decided to add the OpenAI bit here to see if NLP valence measures act differently to a GPT-generaged summary of lyrics than to the lyrics themselves.


In [25]:
##Step 0: Red in data and nltk

df = pd.read_pickle('BeatlesSongsNLP.pkl')

import pandas as pd
import openai
import os

# Set your OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = "foo"
#os.environ["OPENAI_API_ORG"] = "bar"


# Get your OpenAI API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")
# api_org = os.environ.get("OPENAI_API_ORG")

# Print the API key to verify that it was retrieved correctly
#print(api_key)


In [3]:
openai.api_key = api_key
#openai.organization = api_org



In [4]:

def summarize_text(text):
    model_engine = "text-davinci-002"
    prompt = (f"Please summarize the following song lyrics:\n{text}\n\nSummary:")
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.5,
    )
    summary = response.choices[0].text.strip()
    return summary


df["summary"] = df["lyrics"].apply(summarize_text)


In [5]:
print(df["summary"])

0      The lyrics of the song are about the vastness ...
1      The lyrics of the song are about a man who is ...
2      The song is about how easy it is to be with so...
3      The song is about a person who is leaving to g...
4      The lyrics of the song are about counting and ...
                             ...                        
193    The song is urging people to dance to a song t...
194    The song is about a guy who is warning another...
195    The song is about a man who is in love with a ...
196    The lyrics of the song are about a person who ...
197    The lyrics to "Free As A Bird" are about longi...
Name: summary, Length: 198, dtype: object


In [6]:


## NOTE - this uses the older (soon to be depricated) API. I will update this to the new API soon.

def summarize_text(text):
    model_engine = "text-davinci-003"
    prompt = (f"Please summarize the following song lyrics:\n{text}\n\nSummary:")
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.5,
    )
    summary = response.choices[0].text.strip()
    return summary


df["summary2"] = df["lyrics"].apply(summarize_text)

##This ran for almost 20 minutes. It cost about $2. 

In [7]:
df.columns

Index(['Id', 'SongKey', 'song', 'songwriters', 'lead_vocals', 'year', 'cover',
       'Era', 'first_release_date', 'max_key', 'mode', 'avg_danceability',
       'avg_energy', 'avg_loudness', 'avg_speechiness', 'avg_acousticness',
       'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_tempo',
       'avg_duration_ms', 'avg_time_signature', 'Song_y', 'lyrics',
       'avg_popularity', 'core_catalogue_releases', 'Canonical_album',
       'first_vocalist', 'Paul_Song', 'John_Song', 'George_Song', 'Ringo_Song',
       'popularity', 'BB_name', 'BB_artist', 'BB_debut_date',
       'BB_peak_position', 'BB_peak_date', 'BB_weeks_on_chart',
       'RollingStonePosition', 'RobSegment', 'LauraSegment', 'EmilySegment',
       'OliviaSegment', 'BrianSegment', 'JackieSegment', 'vader_neg',
       'vader_neu', 'vader_pos', 'vader_compound', 'roberta_neg',
       'roberta_neu', 'roberta_pos', 'Pipeline_sentiment_label',
       'Pipeline_sentiment_score', 'FinetunedSentiment', 'distilbertsent

In [31]:
##Run on gpt-3.5-turbo. I still don't have access to the gpt-4 model through the API for some reason!!

#THis should work for the entire dataframe under normal circumstances but because the OpenAI API is so slow, I kept getting kicked out.
#I had to break it up into chunks. I did it one year at a time
##Doing this piecewise is a bit of a pain.
#See below:
def summarize_text(lyric):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following lyrics for me: {lyric}. Begin each response with a valence score for the lyrics between -1 and 1. Your response should be begin with the valence score, then a comma, then the summary of the lyrics. Example: '0.80, This is a summary of the lyrics.'"},
        ]
    )

    # Return the content of the message from the assistant
    return response['choices'][0]['message']['content']

# Apply the function to the lyrics column and assign the results to the summary2 column
df["summary3dot5turbo"] = df["lyrics"].apply(summarize_text)





In [12]:
# Create an empty dictionary to store the dataframes
dfs = {}

# Loop through each year from 1960 to 1970
for year in range(1960, 1971):
    # Create a copy of the dataframe for that year
    dfs[year] = df.loc[df['year'] == year, :].copy()

    # Apply the summarize_text function to the lyrics
    dfs[year]["summary3dot5turbo"] = dfs[year]["lyrics"].apply(summarize_text)

# Now I can access each year's dataframe like this:
#df_1964 = dfs[1964]


In [13]:
dfs

{1960: Empty DataFrame
 Columns: [Id, SongKey, song, songwriters, lead_vocals, year, cover, Era, first_release_date, max_key, mode, avg_danceability, avg_energy, avg_loudness, avg_speechiness, avg_acousticness, avg_instrumentalness, avg_liveness, avg_valence, avg_tempo, avg_duration_ms, avg_time_signature, Song_y, lyrics, avg_popularity, core_catalogue_releases, Canonical_album, first_vocalist, Paul_Song, John_Song, George_Song, Ringo_Song, popularity, BB_name, BB_artist, BB_debut_date, BB_peak_position, BB_peak_date, BB_weeks_on_chart, RollingStonePosition, RobSegment, LauraSegment, EmilySegment, OliviaSegment, BrianSegment, JackieSegment, vader_neg, vader_neu, vader_pos, vader_compound, roberta_neg, roberta_neu, roberta_pos, Pipeline_sentiment_label, Pipeline_sentiment_score, FinetunedSentiment, distilbertsentiment, summary, summary2, summary3dot5turbo]
 Index: []
 
 [0 rows x 60 columns],
 1961: Empty DataFrame
 Columns: [Id, SongKey, song, songwriters, lead_vocals, year, cover, Era

In [14]:
# Set year to '1995'
year = '1995'

# Create a dataframe for that year
df_1995 = df.loc[df['year'] == year, :].copy()

# Apply the summarize_text function to the lyrics
df_1995["summary3dot5turbo"] = df_1995["lyrics"].apply(summarize_text)

In [15]:
# Set year to '1996'
year = '1996'

# Create a dataframe for that year
df_1996 = df.loc[df['year'] == year, :].copy()

# Apply the summarize_text function to the lyrics
df_1996["summary3dot5turbo"] = df_1996["lyrics"].apply(summarize_text)



In [16]:
df_1962 = dfs[1962]
df_1963 = dfs[1963]
df_1964 = dfs[1964]
df_1965 = dfs[1965]
df_1966 = dfs[1966]
df_1967 = dfs[1967]
df_1968 = dfs[1968]
df_1969 = dfs[1969]
df_1970 = dfs[1970]
#df_1995 = dfs[1995]
#df_1996 = dfs[1996]






In [19]:
# df_1962.shape
# df_1963.shape
# df_1964.shape
# df_1965.shape
# df_1966.shape
# df_1967.shape
# df_1968.shape
#df_1969.shape
#df_1970.shape
#df_1995.shape
#df_1996.shape







(10, 60)

In [20]:
df.value_counts('year')

year
1968    32
1963    30
1964    30
1965    28
1969    26
1967    23
1966    16
1970    10
1962     1
1995     1
1996     1
dtype: int64

# Time to combine results and join back to the main dataframe

In [20]:
# Create a list of the dataframe names
df_names = ['df_1962', 'df_1963', 'df_1964', 'df_1965', 'df_1966', 'df_1967', 'df_1968', 'df_1969', 'df_1970', 'df_1995', 'df_1996']

# Create a list of the dataframes
dfs = [globals()[name] for name in df_names]

# Select only the columns you want to keep from each dataframe
dfs = [df2[['SongKey', 'summary', 'summary2', 'summary3dot5turbo']] for df2 in dfs]

# Concatenate the dataframes
result = pd.concat(dfs)

# Display the resulting dataframe
print(result)


           SongKey                                            summary  \
110       lovemedo  The song is about a person who is in love and ...   
2    allivegottodo  The song is about how easy it is to be with so...   
3      allmyloving  The song is about a person who is leaving to g...   
8      annagotohim  The song is about a man who is telling his gir...   
11        askmewhy  The lyrics of the song express the speaker's l...   
..             ...                                                ...   
114      maggiemae  The song is about a woman named Maggie Mae who...   
134    oneafter909  The song is about a man begging his lover not ...   
171        twoofus  The song is about two people who are in love a...   
197    freeasabird  The lyrics to "Free As A Bird" are about longi...   
196       reallove  The lyrics of the song are about a person who ...   

                                              summary2  \
110  The song is about asking someone to love them ...   
2    Th

In [21]:
result['gpt4'] = 'TBD'
result = result.rename(columns={'summary': 'text-davinci-002', 'summary2': 'text-davinci-003'})



In [166]:
result

,SongKey,text-davinci-002,text-davinci-003,summary3dot5turbo,gpt4
110,lovemedo,"The text is a song about love and devotion. The singer is asking for love in return, and pledging their own love and fidelity. They express a desire for someone new to love, and say that they would love that person just as much as they love the original subject of the song.","The text is a song about wanting to be loved, expressing that the speaker will always be true and asking the listener to love them. It also expresses the need for someone new to love, someone like the listener.","0.85, The lyrics express a deep and unwavering love for someone, asking them to reciprocate and to be true in their feelings. The singer desires to find someone to love and emphasizes the significance of finding someone similar to the person they currently love. Overall, it conveys a strong longing for love and a plea for it to be returned.",TBD
2,allivegottodo,"The text is about how someone can be there for you whenever you need them, and all you have to do is call them on the phone. They will always come running home, and you can always count on them to be there whenever you need them.","The narrator promises that whenever they want to be together or kiss, they only need to call each other on the phone and they will come running home. The same applies for the other person, and they just need to call on the narrator.","0.70, The lyrics depict a joyful and eager attitude towards being available for someone. The speaker expresses that all they have to do is call the person and they will come running home, showing their willingness to be there whenever needed. It also highlights the importance of communication and how a simple phone call can lead to a reunion or a desired kiss. The lyrics emphasize reciprocity, as the speaker reassures the other person that they will be there whenever they are called upon. Overall, the lyrics convey a positive and eager sentiment towards being there for someone.",TBD
3,allmyloving,The speaker promises to kiss their loved one and then miss them when they have to leave. They say they'll write home every day and send all their love to them.,The speaker expresses their love for someone and promises to write home every day and send all their love to them. They also promise to remain true and hope their dreams will come true.,"0.80, The lyrics are about a strong and loving relationship. The speaker promises to always be true and expresses their desire to be with their loved one. Despite being physically apart, they vow to write and send their love every day, hoping that their dreams will come true. The overall tone is positive and hopeful, showcasing a deep affection and commitment to the other person.",TBD
8,annagotohim,"The speaker is telling Anna that if she wants to be free, she should go with the man she loves more than him. He says that he has been searching for a girl to love him like he loves Anna, but every girl he's ever had has broken his heart and left him sad","Anna asks the narrator to set her free so she can be with someone who loves her more. The narrator expresses his sadness that he hasn't been able to find someone to love him like he loves Anna, and asks her to give back her ring to him before she goes. He will then set her free to be with the other person.","0.30, In these lyrics, the speaker, Anna, is being asked by someone to set them free so they can be with another person who supposedly loves them more. Despite still loving Anna, the speaker encourages her to go with the other person if they truly love her more. The speaker expresses their longing for a love like Anna's, as previous relationships have left them heartbroken. In the end, the speaker requests Anna to return their ring, as a symbol of letting go, and encourages her again to go with the other person.",TBD
11,askmewhy,"The speaker is in love with someone and is telling them why they love them. They love the person because they tell the speaker things that th

In [22]:
##merge the dataframes together
mergeddf = pd.merge(df, result, on='SongKey', how='left')



In [24]:
pd.to_pickle(mergeddf, 'BeatlesSongsNLPwithGPT.pkl')